In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, listdir, stat, chdir
from os.path import dirname, join, isfile, isdir, basename
import pyminizip as pz

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from bulletin import default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.notifica import Notifica
from bulletin.utils import utils
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
backup_path = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'backups')

In [ ]:
# Backup
utils.create_backup(first_name = "backup_notifica_" , level=3)
chdir(current_dir)

In [ ]:


# Download
mb = Metabase(limit=250000)
mb.generate_notifica_query('notifica', where='True', replace=True)
notifica_parts = mb.download_notificacao('notifica', load=True)

In [ ]:


# Leitura
notifica = Notifica()

read_saved = True

if (read_saved):
    notifica.load('all_notifica_raw-new', compress=True)
else:
    notifica.read(notifica_parts)
    notifica.save('all_notifica_raw-new', replace=True, compress=True)


In [ ]:


# Normalização
notifica.normalize()

In [ ]:


# Fix Dtypes
notifica.fix_dtypes()

In [ ]:
notifica.df.shape

In [ ]:
notifica.df.dtypes

In [ ]:


# Definição da melhor data de diagnóstico
analise_dtd = notifica.analise_data_diagnostico() # NOTEBOOK PARALELO CRIADO

In [ ]:


# Salvar o arquivo
notifica.save('notifica-new', replace=True, compress=False)

In [ ]:
# CRIA BACKUP COM AS QUERIES QUE FORAM BAIXADAS

queries_path = join('C:\\', 'SESA', 'covid19datascience-dev', 'input', 'queries', 'tmp')
queries = [ f for f in listdir(queries_path) if isfile(join(queries_path, f)) ]

chdir(queries_path)


pz.compress_multiple(queries, [], join(backup_path, f"download_notifica_queries_{hoje.strftime('%d_%m_%Y')}.zip"), "password_notifica_db_2021", 9)


# REMOVE AS QUERIES BAIXADAS
for files in queries:
    try:
        print('Removing file: ', files)
        remove(join(queries_path, files))
    except:
        pass

In [ ]:
now = pd.to_datetime(datetime.today())
final = pd.to_datetime(datetime.today())
delta = final - timedelta(minutes=agora.minute, hours=agora.hour)
print(f"início: {agora.strftime('%d/%m/%Y')}\t{agora.strftime('%H:%M')}")
print(f"agora: {now.strftime('%d/%m/%Y')}\t{now.strftime('%H:%M')}")
print(f"tempo corrido: {delta.strftime('%H:%M:%S')}")